In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from scipy.integrate import odeint
import scipy.constants as cons
import scipy.linalg as la
from qutip import *

from sympy import *
from sympy.solvers.solveset import linsolve
init_printing(use_unicode=True) # pretty printing

# Three Level System Hamiltonian

## Find Eigenvalues and Eigenvectors of Three Level System with Lambda($\Lambda$) Configuration

$$\hat{H} = \hat{H}_A + \hat{H}_{AL}$$

$$ \hat{H}_A = 
\begin{bmatrix}
E_1 & 0 & 0 \\
0 & E_2 & 0 \\
0 & 0 & E_3
\end{bmatrix}$$

$$ \hat{H}_{AL} = \hbar
\begin{bmatrix}
0 & 0 & \Omega_p/2 \\
0 & -(\Delta_p - \Delta_c) & \Omega_c/2 \\
\Omega_p/2 & \Omega_c/2 & -\Delta_p
\end{bmatrix}$$


In [ ]:
# Numeric Solution

omega_p = 2*np.pi * 10**(-6)    # probe field frequency 1MHz
omega_c = 2 * 2*np.pi * 10**(-6)    # coupling field frequency 2MHz

E_1, E_2, E_3 = 0, 1, 2

H_a = np.array([[E_1, 0, 0], [0, E_2, 0], [0, 0, E_3]])
H_al = np.array([[0, 0, omega_p], [0, 0, omega_c], [omega_p, omega_c, 0]])
H = H_a + H_al 

eigvals, eigvecs = la.eig(H_al)
eigvals

In [ ]:
# Symbolic Solution

omega_p, omega_c, E_1, E_2, E_3, hbar = symbols('Omega_p Omega_c E_1 E_2 E_3 hbar')

H_a = Matrix([[E_1, 0, 0], [0, E_2, 0], [0, 0, E_3]])
H_al = hbar * Matrix([[0, 0, omega_p/2], [0, 0, omega_c/2], [omega_p/2, omega_c/2, 0]])
H = H_a + H_al 

a = H_al.eigenvals(simplify=True)
s = H_al.eigenvects(simplify=True)
s

# Three Level System Optical Bloch Equations

3D Density Matrix:

$$\hat{\rho} = \begin{bmatrix}
\rho_{11} & \rho_{12} & \rho_{13} \\
\rho_{21} & \rho_{22} & \rho_{23} \\
\rho_{31} & \rho_{32} & \rho_{33}
\end{bmatrix}$$

Interaction Hamiltonian:

$$ \hat{H}_{AL} = \hbar
\begin{bmatrix}
0 & 0 & \Omega_p/2 \\
0 & -(\Delta_p - \Delta_c) & \Omega_c/2 \\
\Omega_p/2 & \Omega_c/2 & -\Delta_p
\end{bmatrix}$$

## Lambda Configuration

In [ ]:
# -- Three Level System - Lambda Configuration

# Symbol definitions for density matrix components
rho11, rho12, rho13, rho21, rho22, rho23, rho31, rho32, rho33 = symbols('rho11 rho12 rho13 rho21 rho22 rho23 rho31 rho32 rho33')
delta_p, delta_c = symbols('Delta_p Delta_c')

# Symbol definitions for Hamiltonian components
Omega_p, Omega_c = symbols('Omega_p Omega_c')
hbar = symbols('hbar')

# Symbol definitions for Lindbladian components
gamma1, gamma2 = symbols('gamma_1 gamma_2')

p = Matrix([[rho11, rho12, rho13], [rho21, rho22, rho23], [rho31, rho32, rho33]]) # Density Matrix

H_AL = hbar * Matrix([[0, 0, Omega_p/2], [0, -(delta_p - delta_c), Omega_c/2], [Omega_p/2, Omega_c/2, -delta_p]])

# Atomic Transition Operators
p11 = Matrix([[1, 0, 0], [0, 0, 0], [0, 0, 0]])
p22 = Matrix([[0, 0, 0], [0, 1, 0], [0, 0, 0]])
p33 = Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 1]])
p12 = Matrix([[0, 1, 0], [0, 0, 0], [0, 0, 0]])
p21 = Matrix([[0, 0, 0], [1, 0, 0], [0, 0, 0]])
p13 = Matrix([[0, 0, 1], [0, 0, 0], [0, 0, 0]])
p31 = Matrix([[0, 0, 0], [0, 0, 0], [1, 0, 0]])
p23 = Matrix([[0, 0, 0], [0, 0, 1], [0, 0, 0]])
p32 = Matrix([[0, 0, 0], [0, 0, 0], [0, 1, 0]])

# Lidbladian Components
L31 = gamma1 * ((p13 * p * p31) - (p33 * p)/2 - (p * p33)/2)
L32 = gamma2 * ((p23 * p * p32) - (p33 * p)/2 - (p * p33)/2)

# Lindbladian
L = L31 + L32

# Lindbladian Master Equation
liouville = -(I/hbar) * ((H_AL * p) - (p * H_AL))
obe = liouville + L

cond = obe[0] + obe[4] + obe[8]

if simplify(cond) == 0:
    print("The condition p11 + p22 + p33 = 0 holds")
else:
    print("Conndition does not hold")

#simplify(obe)
simplify(H_AL)

In [ ]:
eqns = [obe[0], obe[4], obe[8], obe[1], obe[2], obe[3], obe[5], obe[6], obe[7], Eq(rho11 + rho22 + rho33, 1)]
vars = [rho11, rho12, rho13, rho21, rho22, rho23, rho31, rho32, rho33]
sol = solve(eqns, vars, rational=False).values()

# Solution of Optical Bloch Equations in Three Level System ($\Lambda$)

## 1. Steady State Solution

In [ ]:
# -- Three Level System - Lambda Configuration Steady State Solution

# Rabi frequencies for probe and coupling fields
Omega_p = 2 * np.pi * 1
Omega_c = 2 * np.pi * 3

wp = (Omega_p*1j)/2
wc = (Omega_c*1j)/2

# Coupling field detuning
dc = 0

# Radiative decay rates
gamma1 = 2 * np.pi * 1
gamma2 = 2 * np.pi * 0

gmean = (gamma1 + gamma2) / 2

# Probe detuning range
dp_range = np.arange(-50,51,0.01)    

sol = np.empty((9, len(dp_range)), dtype='complex')
for dp in range(len(dp_range)):
    
    drho = np.array([[0, 0, wp, 0, 0, 0, -wp, 0, gamma1],                       # p11
                     [0, -(dp_range[dp] - dc)*1j, wc, 0, 0, 0, 0, -wp, 0],      # p12
                     [wp, wc, -dp_range[dp]*1j - gmean, 0, 0, 0, 0, 0, -wp],    # p13
                     [0, 0, 0, (dp_range[dp] - dc)*1j, 0, wp, -wc, 0, 0],       # p21
                     [0, 0, 0, 0, 0, wc, 0, -wc, gamma2],                       # p22
                     [0, 0, 0, wp, wc, -dc*1j - gmean, 0, 0, -wc],              # p23
                     [-wp, 0, 0, -wc, 0, 0, (dp_range[dp]*1j) - gmean, 0, wp],  # p31
                     [0, -wp, 0, 0, -wc, 0, 0, (dc*1j) - gmean, wc],            # p32
                     [0, 0, -wp, 0, 0, -wc, wp, wc, -(gamma1 + gamma2)],        # p33
                     [1, 0, 0, 0, 1, 0, 0, 0, 1]])

    b = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

    rho = la.lstsq(drho, b)[0]
    sol[:, dp] = rho

pop = sol[0] + sol[4] + sol[8]  # total populations

# ---------- Plotting ----------
plt.style.use(['science', 'notebook', 'grid'])

# Populations
plt.plot(dp_range/(2*np.pi), sol[0, :].real, label=r"$|1\rangle$")
plt.plot(dp_range/(2*np.pi), sol[4, :].real, label=r"$|2\rangle$")
plt.plot(dp_range/(2*np.pi), sol[8, :].real, label=r"$|3\rangle$")

# Total population
#plt.plot(dp_range/(2*np.pi), pop.real, label="Total populations")

plt.xlabel('$\Delta_p/2\pi$')
plt.ylabel(r'Population ($\rho$)')

plt.legend(loc='upper right')
#plt.legend(loc='upper right', bbox_to_anchor=(1.35, 0.75))
#plt.savefig('')

### Comparison Between Two-Level and Three-Level System 

In [ ]:
# ---------- Plotting ----------
plt.style.use(['science', 'notebook', 'grid'])

%store -r rho12

# Rho13 Im part
plt.plot(dp_range/(2*np.pi), sol[2, :].imag, label=r'Three-Level', color='tab:blue')
plt.plot(dp_range/(2*np.pi), rho12, label=r'Two-Level', color='tab:red')

plt.xlabel('$\Delta_p/2\pi$')
plt.ylabel('Absorption')

plt.legend(loc='upper right')
#plt.legend(loc='upper right', bbox_to_anchor=(1.35, 0.75))
plt.savefig('Figures/3LS_comparison.pdf')

## 2. Numerical Solution

### a. Lambda Configuration
$$\displaystyle \left[\begin{matrix}- \frac{i \left(- \Omega_{p} \rho_{13} + \Omega_{p} \rho_{31}\right)}{\hbar} & - \frac{i \left(- \Omega_{c} \rho_{13} + \Omega_{p} \rho_{32}\right)}{\hbar} & - \frac{i \left(- \Omega_{c} \rho_{12} - \Omega_{p} \rho_{11} + \Omega_{p} \rho_{33}\right)}{\hbar}\\- \frac{i \left(\Omega_{c} \rho_{31} - \Omega_{p} \rho_{23}\right)}{\hbar} & - \frac{i \left(- \Omega_{c} \rho_{23} + \Omega_{c} \rho_{32}\right)}{\hbar} & - \frac{i \left(- \Omega_{c} \rho_{22} + \Omega_{c} \rho_{33} - \Omega_{p} \rho_{21}\right)}{\hbar}\\- \frac{i \left(\Omega_{c} \rho_{21} + \Omega_{p} \rho_{11} - \Omega_{p} \rho_{33}\right)}{\hbar} & - \frac{i \left(\Omega_{c} \rho_{22} - \Omega_{c} \rho_{33} + \Omega_{p} \rho_{12}\right)}{\hbar} & - \frac{i \left(\Omega_{c} \rho_{23} - \Omega_{c} \rho_{32} + \Omega_{p} \rho_{13} - \Omega_{p} \rho_{31}\right)}{\hbar}\end{matrix}\right]$$

#### Steady State Solution

In [ ]:
 # -- Three Level System - Lambda Configuration Steady State Solution

omega_p = 2*np.pi * 10**(-6)    # probe field frequency 1MHz
omega_c = 2 * (2*np.pi * 10**(-6))    # coupling field frequency 2MHz
gamma1, gamma2 = 1, 1
hbar = cons.hbar

wp = complex(0, omega_p)/hbar
wc = complex(0, omega_c)/hbar

# wrong ordering
#drho = np.array([[0, 0, wp, 0, 0, 0, -wp, 0, gamma1],                   # p11
#                 [0, 0, 0, 0, 0, wc, 0, -wc, gamma2],                   # p22
#                 [0, 0, -wp, 0, 0, -wc, wp, wc, -(gamma1 + gamma2)],    # p33
#                 [0, 0, wc, 0, 0, 0, 0, -wp, 0],                        # p12
#                 [wp, wc, -(gamma1 + gamma2)/2, 0, 0, 0, 0, 0, -wp],    # p13
#                 [0, 0, 0, wp, wc, -(gamma1 + gamma2)/2, 0, 0, -wc],    # p23
#                 [0, 0, 0, 0, 0, wp, -wc, 0, 0],                        # p21
#                 [-wp, 0, 0, -wc, 0, 0, -(gamma1 + gamma2)/2, 0, wp],   # p31
#                 [0, -wp, 0, 0, -wc, 0, 0, -(gamma1 + gamma2)/2, wc]])  # p32

drho = np.array([[0, 0, gamma1, 0, wp, 0, 0, -wp, 0],                    # p11
                 [0, 0, gamma2, 0, 0, wc, 0, 0, -wc],                    # p22
                 [0, 0, -(gamma1 + gamma2), 0, -wp, -wc, 0, wp, wc],     # p33
                 [0, 0, 0, 0, wc, 0, 0, 0, -wp],                         # p12
                 [wp, 0, -wp, wc, -(gamma1 + gamma2)/2, 0, 0, 0, 0],     # p13
                 [0, wc, -wc, 0, 0, -(gamma1 + gamma2)/2, wp, 0, 0]])    # p23
              
#                 [0, 0, 0, 0, 0, wp, 0, -wc, 0],                        # p21
#                 [-wp, 0, wp, 0, 0, 0, -wc, -(gamma1 + gamma2)/2, 0],   # p31
#                 [0, -wc, wc, -wp, 0, 0, 0, 0, -(gamma1 + gamma2)/2]])  # p32

b = np.array([[0], [0], [0], [0], [0], [0], [0], [0], [0]])

#pl, u = la.lu(drho, permute_l=True)
#u
#rho = la.solve(drho, b)

In [ ]:
omega_p, omega_c = symbols('Omega_p Omega_c')

             # p11 p12  p13   p21 p22 p23  p31   p32 p33
drho = Matrix([[0, 0, -omega_p, 0, 0, 0, omega_p, 0, 0],                 # 1
               [0, 0, -omega_c, 0, 0, 0, 0, omega_p, 0],                 # 2
               [-omega_p, -omega_c, 0, 0, 0, 0, 0, 0, omega_p],          # 3
               [0, 0, 0, 0, 0, -omega_p, omega_c, 0, 0],                 # 4
               [0, 0, 0, 0, 0, -omega_c, 0, omega_c, 0],                 # 5
               [0, 0, 0, -omega_p, -omega_c, 0, 0, 0, omega_c],          # 6
               [omega_p, 0, 0, omega_c, 0, 0, 0, 0, -omega_p],           # 7
               [0, omega_p, 0, 0, omega_c, 0, 0, 0, -omega_c],           # 8
               [0, 0, omega_p, 0, 0, omega_c, -omega_p, -omega_c, 0]])   # 9

L, U, _ = drho.LUdecomposition()

U


#### Dynamic Solution

In [ ]:
# -- Three Level System - Lambda Configuration

omega_p = 2*np.pi * 10**(-6)    # probe field frequency 1MHz
omega_c = 2 * (2*np.pi * 10**(-6))    # coupling field frequency 2MHz

t_i = 0
t_f = (10*np.pi) / omega_p
t = np.linspace(t_i, t_f, 350)

def dSdt(t, S):
    c = - 1/cons.hbar
    p11, p12, p13, p21, p22, p23, p31, p32, p33 = S
    return [c * ((p31 - p13) * omega_p),                              # dp11dt
           c * ((p32 * omega_p) - (p13 * omega_c)),                   # dp12dt
           c * ((p33 - p11) * omega_p - (p12 * omega_c)),             # dp13dt 
           c * ((p31 * omega_c) - (p23 * omega_p)),                   # dp21dt
           c * ((p32 * omega_c) - (p23 * omega_c)),                   # dp22dt  
           c * (((p33 - p22) * omega_c) - (p21 * omega_p)),           # dp23dt
           c * (((p11 - p33) * omega_p) + (p21 * omega_c)),           # dp31dt
           c * (((p22 - p33) * omega_c) + (p12 * omega_p)),           # dp32dt
           c * (((p23 - p32) * omega_c) + ((p13 - p31) * omega_p))]   # dp33dt
            
p11_0, p12_0, p13_0 = 1, 0, 0
p21_0, p22_0, p23_0 = 0, 0, 0
p31_0, p32_0, p33_0 = 0, 0, 0
    
S_0 = (p11_0, p12_0, p13_0, p21_0, p22_0, p23_0, p31_0, p32_0, p33_0)

sol = odeint(dSdt, y0=S_0, t=t, tfirst=True)

p11_sol, p12_sol, p13_sol = sol.T[0], sol.T[1], sol.T[2]
p21_sol, p22_sol, p23_sol = sol.T[3], sol.T[4], sol.T[5]
p31_sol, p32_sol, p33_sol = sol.T[6], sol.T[7], sol.T[8]

# Plotting
plt.plot(t, p11_sol) #label=r'|g$\rangle$')
#plt.plot(t, p12_sol) #label=r'|g$\rangle$')
#plt.plot(t, p13_sol) #label=r'|g$\rangle$')
#plt.plot(t, p21_sol) #label=r'|g$\rangle$')
plt.plot(t, p22_sol) #label=r'|g$\rangle$')
#plt.plot(t, p23_sol) #label=r'|g$\rangle$')
#plt.plot(t, p31_sol) #label=r'|g$\rangle$')
#plt.plot(t, p32_sol) #label=r'|g$\rangle$')
plt.plot(t, p33_sol) #label=r'|g$\rangle$')

plt.xlabel("Time")
plt.ylabel("Population")
#plt.legend(loc='upper right')

### b. Ladder Configuration
$$
\displaystyle \left[\begin{matrix}- \frac{i \left(- \Omega_{1} \rho_{12} + \Omega_{1} \rho_{21}\right)}{\hbar} & - \frac{i \left(- \Omega_{1} \rho_{11} + \Omega_{1} \rho_{22} - \Omega_{2} \rho_{13}\right)}{\hbar} & - \frac{i \left(\Omega_{1} \rho_{23} - \Omega_{2} \rho_{12}\right)}{\hbar}\\- \frac{i \left(\Omega_{1} \rho_{11} - \Omega_{1} \rho_{22} + \Omega_{2} \rho_{31}\right)}{\hbar} & - \frac{i \left(\Omega_{1} \rho_{12} - \Omega_{1} \rho_{21} - \Omega_{2} \rho_{23} + \Omega_{2} \rho_{32}\right)}{\hbar} & - \frac{i \left(\Omega_{1} \rho_{13} - \Omega_{2} \rho_{22} + \Omega_{2} \rho_{33}\right)}{\hbar}\\- \frac{i \left(- \Omega_{1} \rho_{32} + \Omega_{2} \rho_{21}\right)}{\hbar} & - \frac{i \left(- \Omega_{1} \rho_{31} + \Omega_{2} \rho_{22} - \Omega_{2} \rho_{33}\right)}{\hbar} & - \frac{i \left(\Omega_{2} \rho_{23} - \Omega_{2} \rho_{32}\right)}{\hbar}\end{matrix}\right]$$

In [ ]:
# -- Three Level System - Ladder Configuration

omega_1 = 2*np.pi * 10**(-6)    # probe field frequency 1MHz
omega_2 = 2 * (2*np.pi * 10**(-6))    # coupling field frequency 2MHz

t_i = 0
t_f = (10*np.pi) / omega_1
t = np.linspace(t_i, t_f, 350)

def dSdt(t, S):
    c = - 1/cons.hbar
    p11, p12, p13, p21, p22, p23, p31, p32, p33 = S
    return [c * ((p21 - p12) * omega_1),                        # dp11dt
           c * ((p22 - p11) * omega_1 - (p13 * omega_2)),       # dp12dt
           c * ((p23 * omega_1) - (p12 * omega_2)),             # dp13dt 
           c * ((p11 - p22) * omega_1 + (p31 * omega_2)),       # dp21dt
           c * ((p12 - p21) * omega_1 + (p32 - p23) * omega_2), # dp22dt  
           c * ((p13 * omega_1) + (p33 - p32) * omega_2),       # dp23dt
           c * ((p21*omega_2) - (p32 * omega_1)),               # dp31dt
           c * ((-p31 * omega_1) + (p22 - p33) * omega_2),      # dp32dt
           c * ((p23 - p32) * omega_2)]                         # dp33dt
            
p11_0, p12_0, p13_0 = 1, 0, 0
p21_0, p22_0, p23_0 = 0, 0, 0
p31_0, p32_0, p33_0 = 0, 0, 0
    
S_0 = (p11_0, p12_0, p13_0, p21_0, p22_0, p23_0, p31_0, p32_0, p33_0)

sol = odeint(dSdt, y0=S_0, t=t, tfirst=True)

p11_sol, p12_sol, p13_sol = sol.T[0], sol.T[1], sol.T[2]
p21_sol, p22_sol, p23_sol = sol.T[3], sol.T[4], sol.T[5]
p31_sol, p32_sol, p33_sol = sol.T[6], sol.T[7], sol.T[8]

# Plotting
plt.plot(t, p11_sol) #label=r'|g$\rangle$')
#plt.plot(t, p12_sol) #label=r'|g$\rangle$')
#plt.plot(t, p13_sol) #label=r'|g$\rangle$')
#plt.plot(t, p21_sol) #label=r'|g$\rangle$')
plt.plot(t, p22_sol) #label=r'|g$\rangle$')
#plt.plot(t, p23_sol) #label=r'|g$\rangle$')
#plt.plot(t, p31_sol) #label=r'|g$\rangle$')
#plt.plot(t, p32_sol) #label=r'|g$\rangle$')
plt.plot(t, p33_sol) #label=r'|g$\rangle$')

plt.xlabel("Time")
plt.ylabel("Population")
#plt.legend(loc='upper right')

## 3. Solution Using Qutip Toolbox

In [ ]:
# -- Three Level System - Lambda Configuration

omega_p = 2*np.pi * 10**(-6)    # probe field frequency 1MHz
omega_c = 2 * (2*np.pi * 10**(-6))    # coupling field frequency 2MHz

t_i = 0
t_f = (10*np.pi) / omega_p
t = np.linspace(t_i, t_f, 350)

ket_1 = basis(3,0)
ket_2 = basis(3,1)
ket_3 = basis(3,2)

rho0 = basis(3,0) * basis(3,0).dag()

H_al = omega_p * ((ket_1 * ket_3.dag()) +  (ket_3 * ket_1.dag())) + omega_c * ((ket_3 * ket_2.dag()) + (ket_2 * ket_3.dag()))

sol = mesolve(H_al, rho0, t, c_ops=None, e_ops=[ket_1 * ket_1.dag(), ket_2 * ket_2.dag(), ket_3 * ket_3.dag()])

sol_1, sol_2, sol_3 = sol.expect[0], sol.expect[1], sol.expect[2]

# Plotting
plt.plot(t, sol_1, label=r'|1$\rangle$')
plt.plot(t, sol_2, 'r--', label=r'|2$\rangle$')
plt.plot(t, sol_3, 'g-.', label=r'|3$\rangle$')
plt.xlabel("Time")
plt.ylabel("Population")
plt.legend(loc='upper right')

### TESTING SUM OF POPULATIONS ###
a = []    # after for loop is executed this list should contain only ones in it.
for i in range(len(sol_1)):
    sum = sol_1[i] + sol_2[i] + sol_3[i]
    a.append(sum)